#### The goal of this is to produce a house value based on data retrieved from multiple data sources

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

## Import data

In [38]:
# Import the price paid dataset from the Land Registry API for the Chester Area

price_df = pd.read_csv('price_paid_CHESTER.csv', index_col = 0)

##### Need to parse the dates to get year/month from datetime

# Concat the postcode and house number to merge on
price_df['postcode_number'] = price_df['PAON']+' '+price_df['Postcode']

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [39]:
price_df.tail()

,url,Price Paid,Transaction Date,Transaction Id,Type,About,PAON,Postcode,Street Name,postcode_number
183,http://landregistry.data.gov.uk/data/ppi/trans...,740000,"Thu, 30 Sep 2021",D4D42645-57A5-27F6-E053-6C04A8C0A572,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,PEAR TREE COTTAGE,CH3 7JY,BARNHOUSE LANE,PEAR TREE COTTAGE CH3 7JY
184,http://landregistry.data.gov.uk/data/ppi/trans...,283000,"Fri, 19 Nov 2021",D4D42645-56F9-27F6-E053-6C04A8C0A572,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,26,CH1 2NE,BRENNUS PLACE,26 CH1 2NE
185,http://landregistry.data.gov.uk/data/ppi/trans...,220000,"Fri, 08 Oct 2021",D4D42646-7DAE-27F6-E053-6C04A8C0A572,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,7,CH4 0PY,BLACKTHORN CLOSE,7 CH4 0PY
186,http://landregistry.data.gov.uk/data/ppi/trans...,364400,"Fri, 18 Jun 2021",D4D42645-57E4-27F6-E053-6C04A8C0A572,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,45,CH2 2PN,ULLSWATER CRESCENT,45 CH2 2PN
187,http://landregistry.data.gov.uk/data/ppi/trans...,210000,"Thu, 30 Sep 2021",D4D42645-54B4-27F6-E053-6C04A8C0A572,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,12,CH3 5DL,LORD STREET,12 CH3 5DL


In [35]:
# Import the EPC data (Only for CH1)

epc_df = pd.read_csv('epc_CH1.csv', index_col = 0)

# Need to construct a field to merge on
# The data formatting is awful...need to use regex to clean up
# This is a quick and dirty attempt...need to study edge cases and try to improve

def clean_data(x):
    if ',' in x:
        return x.split(',')[0]
    else:
        split = x.split(' ')
        for i in split:
            try:
                num = int(split[0])
                return numtreet Name
            except:
                pass
        return split[0]
        

epc_df['house/number'] = epc_df['address1'].apply(lambda x: clean_data(x))
epc_df['postcode_number'] = str(epc_df['house/number'])+' '+epc_df['postcode']


epc_df.head()

,lmk-key,address1,address2,address3,postcode,building-reference-number,current-energy-rating,potential-energy-rating,current-energy-efficiency,potential-energy-efficiency,...,posttown,construction-age-band,lodgement-datetime,tenure,fixed-lighting-outlets-count,low-energy-fixed-light-count,uprn,uprn-source,house/number,postcode_number
0,691200819902019042819015293112888,"7, Tennyson Walk",Blacon,NaN,CH1 5UN,2919111968,A,A,92,94,...,CHESTER,England and Wales: 1967-1975,2019-04-28 19:01:52,owner-occupied,NaN,NaN,2.000008e+11,Address Matched,7,0 7\n1 2\n2 ...
1,692126352812017030820535191030696,"2, Meadowcroft",Saughall,NaN,CH1 6EU,6688211968,A,A,93,96,...,CHESTER,England and Wales: 1976-1982,2017-03-08 20:53:51,owner-occupied,NaN,NaN,1.000100e+11,Address Matched,2,0 7\n1 2\n2 ...
2,288745278612009052014570601910069,Apartment 123 The Quarter,Egerton Street,NaN,CH1 3NR,916812668,A,A,94,96,...,CHESTER,England and Wales: 2003-2006,2009-05-20 14:57:06,owner-occupied,NaN,NaN,1.000008e+10,Address Matched,Apartment,0 7\n1 2\n2 ...
3,b0cde21277dd10b73f076fadd46c051db5c5d84e5a87f0...,3 Tillage Close,Backford,NaN,CH1 6PZ,10001513495,B,B,84,84,...,CHESTER,2020,2021-09-30 14:45:38,Not defined - use in the case of a new dwellin...,10.0,NaN,1.009468e+10,Energy Assessor,3,0 7\n1 2\n2 ...
4,dd374d1e4a1b52185a2b1031e55f74d4cef4770537eb55...,Eden House,Well Lane,Mollington,CH1 6LD,10001642164,B,B,87,91,...,Chester,2021,2021-09-30 10:34:32,Not defined - use in the case of a new dwellin...,100.0,NaN,NaN,NaN,Eden,0 7\n1 2\n2 ...


In [42]:
# Merge the datasets

merged_df = pd.merge(price_df, epc_df, how = 'left')

In [43]:
merged_df.head()

,url,Price Paid,Transaction Date,Transaction Id,Type,About,PAON,Postcode,Street Name,postcode_number,...,constituency-label,posttown,construction-age-band,lodgement-datetime,tenure,fixed-lighting-outlets-count,low-energy-fixed-light-count,uprn,uprn-source,house/number
0,http://landregistry.data.gov.uk/data/ppi/trans...,79000,"Wed, 06 Sep 1995",3CA2890E-E9BA-4516-9B95-00A8C9A9BFCF,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,IVY BANK,CH3 7BA,PLOUGH LANE,IVY BANK CH3 7BA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://landregistry.data.gov.uk/data/ppi/trans...,145000,"Tue, 10 Apr 2001",F98FC94B-3AE1-4EF6-A256-C9313CEA906C,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,IVY BANK,CH3 7BA,PLOUGH LANE,IVY BANK CH3 7BA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://landregistry.data.gov.uk/data/ppi/trans...,98000,"Fri, 17 Oct 1997",6C015BDC-FC24-4C26-ACF4-8DE631113E1A,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,IVY BANK,CH3 7BA,PLOUGH LANE,IVY BANK CH3 7BA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,http://landregistry.data.gov.uk/data/ppi/trans...,195000,"Fri, 21 Feb 2003",42F6C91B-BE07-4F09-80B6-95CCFEE6B448,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,IVY BANK,CH3 7BA,PLOUGH LANE,IVY BANK CH3 7BA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://landregistry.data.gov.uk/data/ppi/trans...,67000,"Fri, 20 Oct 1995",35780D8A-F061-457F-986E-000F170EEB8D,['http://landregistry.data.gov.uk/def/ppi/Tran...,http://landregistry.data.gov.uk/data/ppi/addre...,44,CH4 0LB,WELL HOUSE DRIVE,44 CH4 0LB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
